<a href="https://colab.research.google.com/github/rainzoo/notebook/blob/main/forex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisite

In [10]:
! apt install ghostscript python3-tk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-tk is already the newest version (3.10.8-1~22.04).
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs9 libgs9-common libidn12 libijs-0.35
  libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 10 newly installed, 0 to remove and 15 not upgraded.
Need to get 16.7 MB of archives.
After this operation, 63.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 

In [11]:
!pip install "camelot-py"
!pip install "requests"
!pip install ghostscript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 451.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 27.9 MB/s eta 0:00:00


In [13]:
# Check Ghostscript
from ctypes.util import find_library
find_library("gs")

from google.colab import data_table
data_table.enable_dataframe_formatter()

import camelot
import requests

# Download Data

In [14]:
urls = {
    "sbi": "https://sbi.co.in/documents/16012/1400784/FOREX_CARD_RATES.pdf",
    "hdfc":"https://www.hdfcbank.com/content/bbp/repositories/723fb80a-2dde-42a3-9793-7ae1be57c87f/?path=/Personal/Home/content/rates.pdf",
    }

# Download files

for bank, file_url in urls.items():
    file_data = requests.get(file_url).content
    with open(f"{bank}.pdf", "wb") as file:
        file.write(file_data)


# Extract Data

In [15]:
tables = []
for bank, url in urls.items():
    table = camelot.read_pdf(f'{bank}.pdf')
    tables.append(table[0].df)

In [19]:
tables[0].loc[1:2]

,0,1,2,3,4,5,6,7,8,9
1,CURRENCY,,TT BUY,TT SELL,BILL BUY,BILL SELL,FOREX TRAVEL \nCARD BUY,FOREX TRAVEL \nCARD SELL,CN BUY,CN SELL
2,UNITED STATES DOLLAR,USD/INR,82.47,83.97,82.40,84.14,82.40,84.14,81.45,84.45


In [18]:
tables[1].iloc[[1,21]]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
1,Currency Type,Currency (In \nRs.),Cash Buying,Cash Selling,Bills Buying \n(Trade),Bills \nSelling \n(Trade),T.T. Buying\n(Inw Rem),T.T. \nSelling\n(O/w \nRem),Forex cards \n(Cash out) & \nTC Purchase,Forex cards \n(Load / \nreLoad) & \nTC sale,DD \nIssuance,Interest cost,,,
21,United States Dollar,USD,81.09,85.45,81.45,85.08,82.00,84.62,82.00,84.87,85.08,,,,


# Filter the required Currency

In [20]:
banks = {
    "sbi": [[1, 2], [0, 2]],
    "hdfc": [[1, 21], [0, 6]]
}

def get_rate(bank: str):
    tables = camelot.read_pdf(f'{bank}.pdf')
    df = tables[0].df
    loc = banks[bank]

    usd = None

    if bank == 'sbi':
        usd = df.iloc[[1, 2], [0, 2]]
    elif bank == 'hdfc':
        usd = df.iloc[[1, 21], [0, 6]]
    else:
        raise ValueError("Invalid bank")
    value = float(usd.iloc[1, 1])
    return usd

rates = []
for bank in banks:
    rates.append(get_rate(bank))

In [21]:
rates[0]

,0,2
1,CURRENCY,TT BUY
2,UNITED STATES DOLLAR,82.47


In [22]:
rates[1]

,0,6
1,Currency Type,T.T. Buying\n(Inw Rem)
21,United States Dollar,82.00


# Persist

In [23]:
import duckdb

In [27]:
# Read DataFrame and execute a SQL query
duckdb.register('forex_rates', rates[0])

duckdb.sql(''' SELECT * FROM forex_rates ''').fetchall()

[('CURRENCY', 'TT BUY'), ('UNITED STATES DOLLAR', '82.47')]

In [72]:
# Create a table in the database
with duckdb.connect("rates.db") as conn:
    conn.execute('''CREATE TABLE forex_rates_table (
        date DATE,
        bank VARCHAR,
        currency varchar,
        tt_buy DOUBLE)''')

In [66]:
# Test with dummy data to debug

"""
with duckdb.connect("rates.db") as conn:
    conn.execute('''INSERT INTO forex_rates_table VALUES (current_date, ?, ?, ?) ''',
                 ['sbi', 'usd', 82])
"""

In [78]:
print(rates[0].columns)
print(rates[1].columns)

Int64Index([0, 2], dtype='int64')
Int64Index([0, 6], dtype='int64')


In [75]:
# find the values
print(rates[0][2][2])
print(rates[1][6][21])

82.47
82.00


In [76]:
# Insert data

values = {
   "sbi": rates[0][2][2],
   "hdfc": rates[1][6][21]
}

with duckdb.connect("rates.db") as conn:
    currency = 'usd'
    for bank, rate in values.items():
        conn.execute('''INSERT INTO forex_rates_table VALUES (current_date, ?, ?, ?) ''',
                    [bank, currency, rate ])

In [77]:
# Query Database Rows as DataFrame
result = None
with duckdb.connect("rates.db") as conn:
    result = conn.execute('SELECT * FROM forex_rates_table').fetch_df()
result

,date,bank,currency,tt_buy
0,2023-12-02,sbi,usd,82.47
1,2023-12-02,hdfc,usd,82.00
